# Is a good portion of an ML execution spent in C code?

In [1]:
%run profiler_helper.py

## Acquiring cProfile
We acquire the binary output of [cProfile](https://docs.python.org/3.7/library/profile.html#module-cProfile) by running:

`python -m cProfile -o output.cprof code_i_want_to_profile.py`

In [3]:
# !python -m cProfile -o cifar10/cifar10_cnn.cprof cifar10/cifar10_cnn.py

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.189
[1,  4000] loss: 1.818
[1,  6000] loss: 1.640
[1,  8000] loss: 1.578
[1, 10000] loss: 1.516
[1, 12000] loss: 1.447
[2,  2000] loss: 1.374
[2,  4000] loss: 1.373
[2,  6000] loss: 1.332
[2,  8000] loss: 1.320
[2, 10000] loss: 1.306
[2, 12000] loss: 1.279
Finished Training
Accuracy of the network on the 10000 test images: 53 %


In [3]:
# !python -m cProfile -o cifar10/cifar10_cnn_2.cprof cifar10/cifar10_cnn_2.py

In [6]:
# !python -m cProfile -o mnist/mnist_cnn.cprof mnist/mnist_cnn.py

Processing...
Done!
[1,  2000] loss: 0.317
[1,  4000] loss: 0.109
[1,  6000] loss: 0.082
[1,  8000] loss: 0.072
[1, 10000] loss: 0.071
[1, 12000] loss: 0.061
[1, 14000] loss: 0.054
[2,  2000] loss: 0.050
[2,  4000] loss: 0.036
[2,  6000] loss: 0.041
[2,  8000] loss: 0.035
[2, 10000] loss: 0.040
[2, 12000] loss: 0.035
[2, 14000] loss: 0.037
Finished Training
Accuracy of the network on the 10000 test images: 98 %


In [22]:
!python -m cProfile -o GANs/mnist_gan.cprof GANs/mnist_gan.py

Epoch [0/20], Step [600/600], d_loss: 0.1415, g_loss: 4.7808, D(x): 0.93, D(G(z)): 0.06
Epoch [1/20], Step [600/600], d_loss: 0.2239, g_loss: 4.7034, D(x): 0.92, D(G(z)): 0.07
Epoch [2/20], Step [600/600], d_loss: 1.2165, g_loss: 1.7609, D(x): 0.74, D(G(z)): 0.40
Epoch [3/20], Step [600/600], d_loss: 0.5476, g_loss: 3.0661, D(x): 0.88, D(G(z)): 0.23
Epoch [4/20], Step [600/600], d_loss: 0.1495, g_loss: 3.4262, D(x): 0.96, D(G(z)): 0.08
Epoch [5/20], Step [600/600], d_loss: 0.1148, g_loss: 4.7976, D(x): 0.95, D(G(z)): 0.02
Epoch [6/20], Step [600/600], d_loss: 0.2035, g_loss: 3.9171, D(x): 0.93, D(G(z)): 0.07
Epoch [7/20], Step [600/600], d_loss: 0.1828, g_loss: 4.3329, D(x): 0.96, D(G(z)): 0.09
Epoch [8/20], Step [600/600], d_loss: 0.1564, g_loss: 4.6253, D(x): 0.94, D(G(z)): 0.01
Epoch [9/20], Step [600/600], d_loss: 0.2558, g_loss: 6.1916, D(x): 0.94, D(G(z)): 0.06
Epoch [10/20], Step [600/600], d_loss: 0.1214, g_loss: 4.1723, D(x): 0.98, D(G(z)): 0.08
Epoch [11/20], Step [600/600], 

## Load the cProfile binary into a pandas DataFrame

In [4]:
df = to_df('cifar10/cifar10_cnn.cprof')
# df

## Preprocess plf column to extract the relevant modules from text

Do you see text like `{built-in method torch._C._get_tracing_state}`? I can't use that. I need to transform it so I extract the module name and function name from such strings. The template class below is like a cookie cutter. You press it down on a string and the remainder is the module name and function name (two-tuple).

In [5]:
toby.validate(to_df('cifar10/cifar10_cnn.cprof'))

cifar10/cifar10_cnn.py:31(__init__)
cifar10/cifar10_cnn.py:40(forward)
cifar10/cifar10_cnn.py:1(<module>)
cifar10/cifar10_cnn.py:29(Net)
Validation failed.


In [8]:
toby.validate(to_df('mnist/mnist_cnn.cprof'))

mnist/mnist_cnn.py:10(__init__)
mnist/mnist_cnn.py:17(forward)
mnist/mnist_cnn.py:9(Net)
mnist/mnist_cnn.py:1(<module>)
Validation failed.


In [24]:
toby.validate(to_df('GANs/mnist_gan.cprof'))

GANs/mnist_gan.py:68(denorm)
GANs/mnist_gan.py:73(reset_grad)
GANs/mnist_gan.py:1(<module>)
Validation failed.


## We see calls to C Extensions below

In [11]:
c_df = get_c_df(df)
# c_df

## Approximately what fraction of time was spent in C code?

In [4]:
get_c_fraction('cifar10/cifar10_cnn.cprof')

'48.989%'

In [7]:
get_c_fraction('mnist/mnist_cnn.cprof')

'58.406%'

In [23]:
get_c_fraction('GANs/mnist_gan.cprof')

'55.528%'